In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from pprint import pprint

In [2]:
## Reading the CSVs
movie_titles_df = pd.read_csv('title_merge_ratings_500000.csv')
user_ratings_df = pd.DataFrame(np.load('User_ratings.npy'))

## convert loaded np array to pd dataframe
user_ratings_df[['userID', 'titleID', 'rating', 'date']] = user_ratings_df[0].str.split(',', expand=True)
user_ratings_df = user_ratings_df.drop(0, axis=1)

In [3]:
movie_titles_df

,Unnamed: 0,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,Action,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,12252,tt0034583,movie,Casablanca,1942,102,"['Drama', 'Romance', 'War']",8.5,578015,0,...,0,0,1,0,0,0,0,0,1,0
1,22712,tt0050083,movie,12 Angry Men,1957,96,"['Crime', 'Drama']",9.0,801467,0,...,0,0,0,0,0,0,0,0,0,0
2,25784,tt0054215,movie,Psycho,1960,109,"['Horror', 'Mystery', 'Thriller']",8.5,679732,0,...,0,0,0,0,0,0,0,1,0,0
3,30145,tt0060196,movie,"The Good, the Bad and the Ugly",1966,161,"['Adventure', 'Western']",8.8,769762,0,...,0,0,0,0,0,0,0,0,0,1
4,31798,tt0062622,movie,2001: A Space Odyssey,1968,149,"['Adventure', 'Sci-Fi']",8.3,677625,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,306015,tt6966692,movie,Green Book,2018,130,"['Biography', 'Comedy', 'Drama']",8.2,506565,0,...,0,0,0,0,0,0,0,0,0,0
289,307352,tt7131622,movie,Once Upon a Time in Hollywood,2019,161,"['Comedy', 'Drama']",7.6,761699,0,...,0,0,0,0,0,0,0,0,0,0
290,308733,tt7286456,movie,Joker,2019,122,"['Crime', 'Drama', 'Thriller']",8.4,1321768,0,...,0,0,0,0,0,0,0,1,0,0
291,318986,tt8579674,movie,1917,2019,119,"['Action', 'Drama', 'War']",8.2,607576,1,...,0,0,0,0,0,0,0,0,1,0


In [4]:
#movie_titles_df = movie_titles_df.drop(movie_titles_df[movie_titles_df.titleType.isin(["tvEpisode", "videoGame", "video", "tvSpecial", "tvShort", "tv", "tvSeries", "tvMiniSeries", "tvPilot", "short"])].index)

In [5]:
movie_titles_df = movie_titles_df.drop(['Unnamed: 0', 'titleType', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'], axis=1)

In [36]:
movie_titles_df

,titleID,primaryTitle
0,tt0034583,Casablanca
1,tt0050083,12 Angry Men
2,tt0054215,Psycho
3,tt0060196,"The Good, the Bad and the Ugly"
4,tt0062622,2001: A Space Odyssey
...,...,...
288,tt6966692,Green Book
289,tt7131622,Once Upon a Time in Hollywood
290,tt7286456,Joker
291,tt8579674,1917


In [7]:
user_ratings_df

,userID,titleID,rating,date
0,ur4592644,tt0120884,10,16 January 2005
1,ur3174947,tt0118688,3,16 January 2005
2,ur3780035,tt0387887,8,16 January 2005
3,ur4592628,tt0346491,1,16 January 2005
4,ur3174947,tt0094721,8,16 January 2005
...,...,...,...,...
4669815,ur0581842,tt0107977,6,16 January 2005
4669816,ur3174947,tt0103776,8,16 January 2005
4669817,ur4592639,tt0107423,9,16 January 2005
4669818,ur4581944,tt0102614,8,16 January 2005


In [8]:
user_ratings_df = user_ratings_df.drop('date', axis=1)

In [9]:
user_ratings_df

,userID,titleID,rating
0,ur4592644,tt0120884,10
1,ur3174947,tt0118688,3
2,ur3780035,tt0387887,8
3,ur4592628,tt0346491,1
4,ur3174947,tt0094721,8
...,...,...,...
4669815,ur0581842,tt0107977,6
4669816,ur3174947,tt0103776,8
4669817,ur4592639,tt0107423,9
4669818,ur4581944,tt0102614,8


In [10]:
user_ratings_df_trimmed = user_ratings_df.groupby('userID').filter(lambda x : len(x)>14)

In [39]:
user_ratings_df_trimmed['userID'].nunique()

29521

In [11]:
user_ratings_df_trimmed

,userID,titleID,rating
1,ur3174947,tt0118688,3
4,ur3174947,tt0094721,8
5,ur1162550,tt0114891,9
8,ur3174947,tt0120601,8
10,ur2035667,tt0335245,10
...,...,...,...
4669813,ur0581842,tt0068424,4
4669814,ur3174947,tt0096895,8
4669815,ur0581842,tt0107977,6
4669816,ur3174947,tt0103776,8


In [12]:
movie_titles_df.rename(columns={'tconst': 'titleID'}, inplace=True)

In [13]:
movie_titles_df

,titleID,primaryTitle
0,tt0034583,Casablanca
1,tt0050083,12 Angry Men
2,tt0054215,Psycho
3,tt0060196,"The Good, the Bad and the Ugly"
4,tt0062622,2001: A Space Odyssey
...,...,...
288,tt6966692,Green Book
289,tt7131622,Once Upon a Time in Hollywood
290,tt7286456,Joker
291,tt8579674,1917


In [14]:
combined_df = pd.merge(movie_titles_df, user_ratings_df_trimmed)

In [15]:
combined_df

,titleID,primaryTitle,userID,rating
0,tt0034583,Casablanca,ur2375356,10
1,tt0034583,Casablanca,ur3212364,9
2,tt0034583,Casablanca,ur2615539,10
3,tt0034583,Casablanca,ur4862994,10
4,tt0034583,Casablanca,ur1293485,10
...,...,...,...,...
155232,tt8946378,Knives Out,ur110879431,8
155233,tt8946378,Knives Out,ur23827727,1
155234,tt8946378,Knives Out,ur109019708,9
155235,tt8946378,Knives Out,ur94666240,10


In [16]:
combined_df.userID.isnull().values.any()

False

In [17]:
combined_df['titleID'].nunique()

287

In [18]:
movie_titles_df['titleID'].nunique()

293

In [19]:
user_ratings_df['titleID'].nunique()

351109

In [20]:
refined_dataset = combined_df.groupby(by=['titleID','userID','primaryTitle'], as_index=False).agg({"rating":"mean"})

In [21]:
refined_dataset

,titleID,userID,primaryTitle,rating
0,tt0034583,ur0011104,Casablanca,1.0
1,tt0034583,ur0011762,Casablanca,10.0
2,tt0034583,ur0012295,Casablanca,7.0
3,tt0034583,ur0013691,Casablanca,10.0
4,tt0034583,ur0016344,Casablanca,10.0
...,...,...,...,...
154285,tt8946378,ur9927546,Knives Out,8.0
154286,tt8946378,ur99622833,Knives Out,9.0
154287,tt8946378,ur99708365,Knives Out,7.0
154288,tt8946378,ur99877336,Knives Out,8.0


In [22]:
movieUser_df = refined_dataset.pivot_table(
    index='userID',
     columns='titleID',
      ## Replacing all movies users haven't rated with a rating of 0
      values='rating').fillna(0)

In [23]:
movieUser_df

titleID,tt0034583,tt0050083,tt0054215,tt0060196,tt0062622,tt0066921,tt0068646,tt0071562,tt0071853,tt0073195,...,tt5463162,tt6320628,tt6644200,tt6723592,tt6751668,tt6966692,tt7131622,tt7286456,tt8579674,tt8946378
userID,,,,,,,,,,,,,,,,,,,,,
ur0000011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
ur0000066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ur99955002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur99964320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
ur99965244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0


In [24]:
movieUser_scipy_df = csr_matrix(movieUser_df.values)

In [25]:
movieUser_scipy_df

<21177x287 sparse matrix of type '<class 'numpy.float64'>'
	with 154290 stored elements in Compressed Sparse Row format>

In [26]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(movieUser_scipy_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [27]:
def similar_users(user, n = 5):
    knn_input = np.asarray([movieUser_df.values[user-1]])
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
    
    print("Top ", n ," users who are most similar to the user #",user, " are:", sep="")
    print("____________________________________________________________________")
    
    
    for i in range(1,len(distances[0])):
        print(i,". User #", indices[0][i]+1, " separated by distance of ",distances[0][i],sep="")
    return indices.flatten()[1:] + 1, distances.flatten()[1:]

In [28]:
similar_user_list, distance_list = similar_users(7, 1)

Top 1 users who are most similar to the user #7 are:
____________________________________________________________________
1. User #28 separated by distance of 0.0


In [29]:
weighted_list = distance_list/np.sum(distance_list)

C:\Users\Fitzi\AppData\Local\Temp\ipykernel_30380\697899973.py:1: RuntimeWarning: invalid value encountered in true_divide
  weighted_list = distance_list/np.sum(distance_list)


In [30]:
similar_user_ratings = movieUser_df.values[similar_user_list]

In [31]:
movie_list = movieUser_df.columns

In [32]:
weighted_list = weighted_list[:,np.newaxis] + np.zeros(len(movie_list))

In [33]:
ratings_matrix = weighted_list*similar_user_ratings
mean_ratings_list = ratings_matrix.sum(axis =0)

In [34]:
def recommend_movies(n):
  n = min(len(mean_ratings_list),n)
  pprint(list(movie_list[np.argsort(mean_ratings_list)[::-1][:n]]))

In [35]:
#target_user = 6771

target_user = 553318 
target_user_id = movieUser_df.index.values[target_user -1]
no_of_similar_users = 7
no_of_highest_rated_movies_by_target_user = 5
no_of_movies_to_recommend = 5

print(" The ",no_of_highest_rated_movies_by_target_user," movies highest rated by user '",target_user_id,"' are:",sep="")
print("____________________________________________________________________")
pprint(list(refined_dataset[refined_dataset['userID'] == target_user_id].sort_values('rating', ascending=False)['titleID'])[:no_of_highest_rated_movies_by_target_user])
print("")
print("")

similar_user_list2, distance_list2 = similar_users(target_user, no_of_similar_users)
weighted_list2 = distance_list2/np.sum(distance_list2)
similar_user_ratings2 = movieUser_df.values[similar_user_list2]
weighted_list2 = weighted_list2[:,np.newaxis] + np.zeros(len(movie_list))
ratings_matrix2 = weighted_list2*similar_user_ratings2
mean_ratings_list2 = ratings_matrix2.sum(axis =0)

def recommend_movies2(n):
  n = min(len(mean_ratings_list2),n)
  pprint(list(movie_list[np.argsort(mean_ratings_list2)[::-1][:n]]))

print("")
print("")
print("Movies most recommended based on these similar users are: ")
print("____________________________________________________________________")
recommend_movies2(no_of_movies_to_recommend)

IndexError: index 553317 is out of bounds for axis 0 with size 21177

In [ ]:
pprint(list(refined_dataset[refined_dataset['userID'] == target_user].sort_values('rating', ascending=False)['primaryTitle'])[:no_of_highest_rated_movies_by_target_user])

In [ ]:
pprint(list(refined_dataset[refined_dataset['userID'] == target_user].sort_values('rating', ascending=False)['primaryTitle']))

In [ ]:
movieUser_df

In [ ]:
movieUser_df.index.values[553318 -1]

In [ ]:
movieUser_df.loc['ur99998429']

In [ ]:
movieUser_df

In [ ]:
list(refined_dataset[refined_dataset['userID'] == target_user].sort_values('rating', ascending=False)['primaryTitle'])[:no_of_highest_rated_movies_by_target_user]

In [ ]:
refined_dataset[refined_dataset['userID'] == 'ur99998429']

In [ ]:
movieUser_df[(movieUser_df.index == "ur99998429")]

In [ ]:
movieUser_df

In [ ]:
#movieUser_df.to_csv('IMDbMovieUserPivot.csv')

In [ ]:
#movieUser_df.to_csv('IMDbMovieUserPivotTrimmed.csv')

In [ ]:
#from joblib import Parallel, delayed
#import joblib

In [ ]:
#joblib.dump(knn_model, 'IMDbModel.h5')

In [ ]:
# refined_dataset.to_csv('IMDbRefined2.csv')